# Mixed volume computations with Oscar

There are many ways to compute mixed volumes for Oscar objects. Feel free to pick the one that feels most natural to you!

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.13.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


## Method 1: The inclusion-exclusion formula

There is no immediately accessible Oscar function for computing mixed volumes. However, we can easilly write our own, by using the formula
$$\operatorname{MV}(P_1,\ldots,P_n)=\sum_{\emptyset\neq I\subseteq\{1,\ldots,n\}} (-1)^{n-|I|}\operatorname{Vol}\Big(\sum_{i\in I} P_i\Big).$$
This is by no means the fastest method for computing mixed volumes, but it will work well for most of our purposes.

In [2]:
# Define the polytopes by their vertices (remember that the rows are interpreted as vertices)
P = convex_hull([0 0; 1 0; 0 1])
Q = convex_hull([0 0; 2 0; 0 2])

Polyhedron in ambient dimension 2

In [3]:
volume(P)

1//2

In [4]:
volume(P+Q)-volume(P)-volume(Q)

2

**Coding trick:** It's often better to use the `import` command instead of the `using` command, if you only need one or a handful of functions from a package. (If you write `using`, you automatically load the whole package!)

In [ ]:
import Combinatorics: powerset

In [5]:
function my_mixed_volume(list_of_polytopes)
    n = length(list_of_polytopes)
    return sum([(-1)^(n-length(I))*volume(sum(list_of_polytopes[I])) for I in powerset(collect(1:n)) if !isempty(I)])
end;

In [6]:
my_mixed_volume([P,Q])

2

In [7]:
R, (x,y) = polynomial_ring(QQ,["x","y"])
f = 1 + x + y
g = 1 + x^2 + y^2
my_mixed_volume([newton_polytope(f),newton_polytope(g)])

2

In [8]:
F = [f,g]
my_mixed_volume(newton_polytope.(F))

2

## Method 2: The Oscar interface to Polymake

Under the hood, Oscar has access to a very powerful software package for polyhedral geometry called Polymake, which in particular includes a fast mixed volume function. We can reach it from Oscar by calling `Polymake.polytope.mixed_volume`. For this to work, you also need to convert your polytopes to Polymake objects by writing `.pm_poltytope`.

In [ ]:
P.pm_polytope

In [ ]:
Polymake.polytope.mixed_volume(P.pm_polytope,Q.pm_polytope)

## Method 3: The `mixed_volume` command from HC.jl

There is also a very fast function for computing mixed volumes in the HomotopyContinuation.jl package that we can use.

**Warning:** Don't import all of the HomotopyContinuation.jl in a notebook where you are using Oscar! There will likely be lots of confusing conflicts.

Intead, use the `import` command to only import the `mixed_volume` command from HomotopyContinuation.jl.

In [9]:
import HomotopyContinuation: mixed_volume

There are a few things to be aware of:

* This command wants a V-representation of the polytopes as input.
* The V-representation should be in the form of an **integer matrix** where the **columns** correspond to the vertices.

**Coding trick:** Use `...` to turn a list into a sequence. Combined with `hcat`, this can turn list of vectors into matrices.

In [10]:
# Example
list_of_vectors = [[0,0],[1,0],[0,1]]
hcat(list_of_vectors...)

2×3 Matrix{Int64}:
 0  1  0
 0  0  1

In [11]:
vertices(P)

3-element SubObjectIterator{PointVector{QQFieldElem}}:
 [0, 0]
 [1, 0]
 [0, 1]

In [12]:
# Converts a rational vector to an integer vector
integer_vector = v -> Int.(Vector(v));

In [13]:
integer_vector.(vertices(P))

3-element Vector{Vector{Int64}}:
 [0, 0]
 [1, 0]
 [0, 1]

In [14]:
# Returns the vertices of a lattice polytope as the *columns* of an integer matrix
vertex_matrix = P::Polyhedron -> hcat( integer_vector.(vertices(P))... )

#9 (generic function with 1 method)

In [15]:
vertex_matrix(Q)

2×3 Matrix{Int64}:
 0  2  0
 0  0  2

In [16]:
mixed_volume([vertex_matrix(P),vertex_matrix(Q)])

2

In [17]:
R, (x,y) = polynomial_ring(QQ,["x","y"])
f = 1+x+y
g = 1+x^2+y^2

x^2 + y^2 + 1

In [18]:
support_matrix = f ->  hcat(collect(exponents(f))...);

In [19]:
support_matrix(f)

2×3 Matrix{Int64}:
 1  0  0
 0  1  0

In [20]:
support_matrix(g)

2×3 Matrix{Int64}:
 2  0  0
 0  2  0

In [21]:
mixed_volume([support_matrix(f),support_matrix(g)])

2

In [22]:
F = [f,g]
mixed_volume(support_matrix.(F))

2